Si queremos simular un sistema de varios cuerpos, la mejor manera de implementar la dinámica entre los elementos será a través de una nueva clase, la cual se encargará de calcular las fuerzas entre los cuerpos y sumarlas a cada uno. Este tipo de clases se conocen como **Colisionadores** o **Interactuadores**. Utilizando el código que desarrollamos en C++ del método de PEFRL (*Métodos de Simulación - Elementos discretos*):

In [ ]:
%%writefile Planetas.cpp

#include <iostream>
#include <cmath>
#include "vector.h"
using namespace std;


// Constantes del problema físico:
const double G=1;
const int N=2;  // Número de cuerpos que voy a crear

// Constantes del algoritmo de integración
const double Xi = 0.1786178958448091;
const double Lambda = -0.2123418310626054;
const double Chi = -0.06626458266981849;
const double Um2LambdaU2 = (1-2*Lambda)/2;
const double Um2ChipXi = 1-2*(Chi+Xi);

// Declaración de las clases:
class Cuerpo;
class Colisionador;

// Declaración de las interfases de las clases:
class Cuerpo{
  private:
    vector3D r,V,F; double m,R;
  public:
    void Inicie(double x0,double y0,double z0,double Vx0,double Vy0,double Vz0,
	        double m0,double R0);
    void BorreFuerza(void){F.load(0,0,0);}; // Inline
    void SumeFuerza(vector3D dF){F+=dF;}; // Inline
    void Mueva_r(double dt, double coef);
    void Mueva_v(double dt, double coef);
    void Dibujese(void);
    double Getx(void){return r.x();}; // Inline
    double Gety(void){return r.y();}; // Inline
    friend class Colisionador;    // De esta manera, hacemos que la clase Colisionador pueda tener acceso a los datos privados de la clase Cuerpo
};
class Colisionador{
  private:
  public:
    void CalculeTodasLasFuerzas(Cuerpo * Planetas);
    void CalculeFuerzaEntre(Cuerpo & Planeta1,Cuerpo & Planeta2);
};

// Implementación de las funciones de las clases:
// Funciones de la clase Cuerpo:
void Cuerpo::Inicie(double x0,double y0,double z0,double Vx0,double Vy0,double Vz0,
	      double m0,double R0){
  r.load(x0,y0,z0); V.load(Vx0,Vy0,Vz0); m=m0; R=R0;
}
void Cuerpo::Mueva_r(double dt, double coef){
  // Algoritmo de Forest-Ruth
  r+=(coef*dt)*V;
}
void Cuerpo::Mueva_v(double dt, double coef){
  // Algoritmo de Forest-Ruth
  V+=(coef*dt/m)*F;
}
void Cuerpo::Dibujese(void){
  cout<<" , "<<r.x()<<"+"<<R<<"*cos(t),"<<r.y()<<"+"<<R<<"*sin(t)";
}

// Funciones de la clase Colisionador:
void Colisionador::CalculeTodasLasFuerzas(Cuerpo * Planeta){
  int i,j;
  // 1) Borramos las fuerzas de todos los planetas:
  for(i=0;i<N;i++){
    Planeta[i].BorreFuerza();
  }
  // 2) Se recorre por parejas la clase Cuerpo creada, se calcula la fuerza de cada pareja y se suma a cada planeta:
  for(i=0;i<N;i++){
    for(j=0;j<i;j++){
      CalculeFuerzaEntre(Planeta[i],Planeta[j]);
    }
  }

}
void Colisionador::CalculeFuerzaEntre(Cuerpo & Planeta1,Cuerpo & Planeta2){
  double m1 = Planeta1.m, m2 = Planeta2.m;
  vector3D r21 = Planeta2.r-Planeta1.r; double r2 = r21.norm2();
  double aux = G*m1*m2*pow(r2,-1.5);
  vector3D F1 = aux*r21;
  Planeta1.SumeFuerza(F1);  Planeta2.SumeFuerza(F1*(-1));
}

//----------- Funciones Globales -----------

//Funciones de animación:
void InicieAnimacion(void){
  //cout<<"set terminal gif animate"<<end;
  //cout<<"set output 'PlanetaFR.gif'"<<end;
  cout<<"unset key"<<endl;
  cout<<"set xrange[-11:11]"<<endl;
  cout<<"set yrange[-11:11]"<<endl;
  cout<<"set size ratio -1"<<endl;
  cout<<"set parametric"<<endl;
  cout<<"set trange[0:7]"<<endl;
  cout<<"set isosamples 12"<<endl;
}

void InicieCuadro(void){
  cout<<"plot 0,0 ";
}

void TermineCuadro(void){
  cout<<endl;
}

int main(){
  double r=11, m0=10, m1=1;
  double M=m0+m1, mu=m0*m1/M;
  double x0=-m1*r/M, x1=m0*r/M;
  double omega=sqrt(G*M/(r*r*r));  double T=2*M_PI/omega;
  double V0=omega*x0, V1=omega*x1;
  double t, dt=0.1, ttotal=10*T;
  int Ncuadros=20000; double tdibujo,tcuadro=ttotal/Ncuadros;
  Cuerpo Planeta[N];
  Colisionador Newton;
  int i;
  InicieAnimacion();
  //----------(x0,y0,z0,Vx0,Vy0,Vz0,m0,R0)
  // INICIO:
  Planeta[0].Inicie(x0, 0, 0, 0, 0.5*V0, 0, m0, 1.0);
  Planeta[1].Inicie(x1, 0, 0, 0, 0.5*V1, 0, m1, 0.5);
  // CORRE PROGRAMA:
  for(t=tdibujo=0;t<ttotal;t+=dt,tdibujo+=dt){
    if(tdibujo>tcuadro){
      //cout<<Planeta[0].Getx()<<" "<<Planeta[0].Gety()<<endl;
      //cout<<Planeta[1].Getx()<<" "<<Planeta[1].Gety()<<endl;
      InicieCuadro();
      for(i=0;i<N;i++) Planeta[i].Dibujese();
      TermineCuadro();
      tdibujo=0;
    }
    /* Ahora el cálculo de las fuerzas depende de la clase colisionador y no de la clase cuerpo. Además, recordemos que estamos
       aplicando el método de integración PEFRL de manera simultánea para los N cuerpos que tengamos.*/
    for(i=0;i<N;i++) Planeta[i].Mueva_r(dt,Xi);
    Newton.CalculeTodasLasFuerzas(Planeta);  for(i=0;i<N;i++) Planeta[i].Mueva_v(dt,Um2LambdaU2);
    for(i=0;i<N;i++) Planeta[i].Mueva_r(dt,Chi);
    Newton.CalculeTodasLasFuerzas(Planeta);  for(i=0;i<N;i++) Planeta[i].Mueva_v(dt,Lambda);
    for(i=0;i<N;i++) Planeta[i].Mueva_r(dt,Um2ChipXi);
    Newton.CalculeTodasLasFuerzas(Planeta);  for(i=0;i<N;i++) Planeta[i].Mueva_v(dt,Lambda);
    for(i=0;i<N;i++) Planeta[i].Mueva_r(dt,Chi);
    Newton.CalculeTodasLasFuerzas(Planeta);  for(i=0;i<N;i++) Planeta[i].Mueva_v(dt,Um2LambdaU2);
    for(i=0;i<N;i++) Planeta[i].Mueva_r(dt,Xi);
  }
  return 0;
}

Overwriting Planetas.cpp


In [ ]:
%%shell
g++ Planetas.cpp
./a.out > "Instrucciones - Planetas.txt"